In [46]:
!pip show sqlalchemy

Name: SQLAlchemy
Version: 2.0.30
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: typing-extensions
Required-by: 


In [47]:

!pip install mysql-connector-python

In [48]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass
import os
from dotenv import load_dotenv
import mysql.connector

In [49]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass

password = getpass.getpass("Please enter the SQL database password: ")
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

# Close the connection properly
connection.close()

In [50]:
from sqlalchemy import text

def rentals_month(engine, month, year):
    query = f"SELECT * FROM rental WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}"
    with engine.connect() as connection:
        result = connection.execute(text(query))
        df = pd.DataFrame(result.fetchall())
    return df

may = rentals_month(engine, 5, 2005)
print(may.head())

   rental_id         rental_date  inventory_id  customer_id  \
0          1 2005-05-24 22:53:30           367          130   
1          2 2005-05-24 22:54:33          1525          459   
2          3 2005-05-24 23:03:39          1711          408   
3          4 2005-05-24 23:04:41          2452          333   
4          5 2005-05-24 23:05:21          2079          222   

          return_date  staff_id         last_update  
0 2005-05-26 22:04:30         1 2006-02-15 21:30:53  
1 2005-05-28 19:40:33         1 2006-02-15 21:30:53  
2 2005-06-01 22:12:39         1 2006-02-15 21:30:53  
3 2005-06-03 01:43:41         2 2006-02-15 21:30:53  
4 2005-06-02 04:33:21         1 2006-02-15 21:30:53  


In [51]:
import pandas as pd

def rental_count_month(df, month, year):
    rentals_by_customer = df.loc[(df['rental_date'].dt.month == month) & (df['rental_date'].dt.year == year)] \
                            .groupby('customer_id')['rental_id'].count().reset_index()
    rentals_by_customer.rename(columns={'rental_id': f'rentals_{month:02d}_{year}'}, inplace=True)
    return rentals_by_customer

In [52]:
def compare_rentals(df1, df2):
    merged_df = pd.merge(df1, df2, on='customer_id', suffixes=('_df1', '_df2'))
    merged_df['difference'] = merged_df[df1.columns[1]] - merged_df[df2.columns[1]]
    return merged_df

# Usage
may_rentals = rentals_month(engine, 5, 2005)
june_rentals = rentals_month(engine, 6, 2005)
may_count = rental_count_month(may_rentals, 5, 2005)
june_count = rental_count_month(june_rentals, 6, 2005)
comparison = compare_rentals(may_count, june_count)
print(comparison.head())

   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1                2                7          -5
1            2                1                1           0
2            3                2                4          -2
3            5                3                5          -2
4            6                3                4          -1
